# OR calculations

Author: Scott Pew

This notebook is written in R and requires a few inputs:

* Control and case counts
* Population frequency of disease
* Odds ratio threshold
* Output from the three workflows
* Functional threshold score cutoffs


In [ ]:
##Version 1.0.
## still need to adjust OR by ethnicity

In [1]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


This script takes case control observational data as input and computes odds ratios
for user defined analytic subsets. It also calculates the proportion of variants
that are pathogenic in each subset.

In [3]:
#Lines 104-118 set user defined thresholds
#Lines 121-209 do computations

## define functions

In [7]:
#functions for proportion pathogenic calculation
probfromrelativerisk = function(delta0, rho)
{
  rho * delta0
}

probfromoddsratio = function(delta0, phi)
{
  phi * delta0 / ( 1 + delta0 * (phi-1))
}

delta = function(p,l,d0,d1)
{
  d0 * exp(-p*l) + d1*(1-exp(-p*l))
}

alpha = function(p,l,d0,d1)
{
  1 - (1-d0)*exp(-l) / (1-delta(p,l,d0,d1))
}

beta = function(p,l,d0,d1)
{
  1 - d0*exp(-l) / delta(p,l,d0,d1)
}

lambda = function(a,b,d0)
{
  log( ((1-b)*(1-d0) + (1-a)*d0) / (1-a)/(1-b))
}

pp = function(a,b,d0,d1)
{
  p = (d1-d0) * ((1-b)*(1-d0) + (1-a)*d0)
  p = p / ( (1-b)*d1*(1-d0) - (1-a)*d0*(1-d1) )
  log(p) / lambda(a,b,d0)
}

propallpath = function(x,n,y,m,d0,d1)
{
  p = pp(x/n,y/m,d0,d1)
  p[p<0] = 0
  p[p>1] = 1
  p
}


propobspath = function(x,n,y,m,d0,d1)
{
  p = propallpath(x,n,y,m,d0,d1)
  l = lambda(x/n,y/m,d0)
  d = delta(p,l,d0,d1)
  
  ai = (p * (1-d1) + (1-p) * (1-d)) / (1-d)
  bi = (p * d1 + (1-p)*d) / d 
  pobs = n * ai + m * bi
  aip = p * (1-d1) /(1-d)
  bip = p * d1 / d
  ppathobs = n*aip + m * bip
  ppathobs/pobs
}

bootall = function(x,n,y,m,d0,d1,b=1000)
{
  xjs = rbinom(b,n,x/n)
  yjs = rbinom(b,m,y/m)
  propallpath(xjs,n,yjs,m,d0,d1)
}

bootobs = function(x,n,y,m,d0,d1,b=1000)
{
  xjs = rbinom(b,n,x/n)
  yjs = rbinom(b,m,y/m)
  propobspath(xjs,n,yjs,m,d0,d1)
}

loglike = function(x,n,y,m,p,l,d0,d1)
{
  a = alpha(p,l,d0,d1)
  b = beta(p,l,d0,d1)
  x * log(a) + (n-x)*log(1-a) + y * log(b) + (m-y) * log(1-b)
}

llplot = function(x,n,y,m,d0,d1)
{
  phat = propallpath(x,n,y,m,d0,d1)
  allp = (0:100)/100
  lhat = lambda(x/n,y/m,d0)
  ll = loglike(x,n,y,m,allp,lhat,d0,d1)
  xl = paste("MLE for p is ",round(100*phat,2),"%",sep="")
  plot(allp,ll,type="l",xlab=xl,ylab="Log likelihood")
  lines(c(phat,phat),range(ll),col=2)
}

## set user defined thresholds

### define functional thresholds

ACTION: Input the loss of function score threshold and functional score threshold from your functional data below: 

NOTE: If your threshold(s) are negative, put them in parentheses.

In [2]:
#loss_of_function_threshold = 0.9
#functional_threshold = 0.1

#must use parentheses for negative thresholds 

loss_of_function_threshold = (-1.328)
functional_threshold = (-0.748)

In [3]:
#check that the sign of your input is correct

sign(loss_of_function_threshold )

[1] -1

### define total case and control counts

ACTION: Input the total number of cases and controls in your observational data below.

In [7]:
#control = 807162
#case = 100000

In [5]:
control = 43481
case = 46306

### define population frequency of disease and odds ratio thresholds

ACTION: Input the values for:
d0 and oddsrat below. 


d0 = population frequency of disease 

oddsrat = odds ratio threshold


We will use them to calculate the probability using the equation:

![image!](https://latex.codecogs.com/svg.latex?\Large&space;d1=\frac{\Phi\cdot\Delta_{0}}{1+\Delta_{0}\left(\Phi-1\right)})

or


![image!](https://latex.codecogs.com/svg.latex?\Large&space;d1=\frac{ORd_{0}}{1+d_{0}\left(OR-1\right)})


In [8]:
d0 = 0.12
oddsrat = 5
d1 = probfromoddsratio(d0,oddsrat)

In [9]:
d1

[1] 0.4054054

### define location of file and import

Input your file with the calibraton variants and functional data and observational data merged. Make sure you have already uploaded it to your workspace data folder.

Upload your own data to your workspace bucket by going to the workspace Data tab, selecting Files, and clicking the upload icon.

To access these files from this Notebook, we use the gsutil command. For this command, you need to specify the Google bucket location (IRL starting with "gs://..."). If you are using your own workspace bucket instead of the public bucket, you can find the Google bucket id on the right side of the workspace dashboard.


In [9]:
# Access counts file in your workspace bucket
# Note that you need to change to your workspace bucket path...

input_data = system(paste0("gsutil cp gs://fc-1de68429-527a-4c60-a614-836b141b502f/BRCA1_calibration_fxnl_case_control.tsv ."))

# Confidence check - list all files in the notebook directory to verify that the file is there
list.files()

[1] "BRCA1_calibration_fxnl_case_control.tsv"    
 [2] "BRCA1_case_control_data.tsv"                
 [3] "BRCA1_fxn_vars_calibration_merged.csv"      
 [4] "BRCA1_fxn_vars_calibration_merged.tsv"      
 [5] "BRCA1_fxn_vars_calibration_short_weird.csv" 
 [6] "BRCA1_variants_clinical_orig_numbering.csv" 
 [7] "BRCA2_case_control_data.tsv"                
 [8] "Bring Your Own Data Tutorial.ipynb"         
 [9] "Clinvar_Jupyter_Notebook.ipynb.md"          
[10] "ClinVar_variants_scrape.ipynb"              
[11] "ClinVar_variants_with_diseases.ipynb"       
[12] "gnomad_Hail_variants_scrape.ipynb"          
[13] "gnomAD-with-Hail(pre-written example).ipynb"
[14] "HGVS_variant_names.ipynb"                   
[15] "OR_Calculations_MESSY_DRAFT_ALLISON.ipynb"  
[16] "OR_Calculations.ipynb"                      
[17] "reformatting_variant_ids_notebook.ipynb"    
[18] "sorting_combined_variants.ipynb"

Now, input the string with your file name below

In [10]:
#import data
input_data =  read.table(file = 'BRCA1_calibration_fxnl_case_control.tsv', sep = '\t',quote="", header = TRUE, fill = TRUE, stringsAsFactors = FALSE)


In [11]:
head(input_data)


,X,hgvs_nt,txpt_hgvsc_from_ID_clinvar,hgvs_pro_clinvar,hgvs_pro_gnomad,txpt_hgvsp_gnomad,variant_source,set_gnomad,n_alt_alleles_exomes_gnomad,n_alt_alleles_genomes_gnomad,⋯,Suggested_ACMG_AMP_evidencef,Logistic_Regression_PS4_criteriong,chr_y,pos_y,ref_y,alt_y,DISTANCE,Consequence,Review_Status,Class
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,⋯,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>
1,0,c.1A>C,c.1A>C (p.Met1Leu),p.Met1Leu,,,ClinVar only,,NA,NA,⋯,,,NA,NA,,,NA,,,
2,1,c.1A>G,c.1A>G (p.Met1Val),p.Met1Val,p.Met1?,ENSP00000350283.3:p.Met1?,gnomAD and ClinVar,exomes,2,NA,⋯,No evidence*,,17,43124096,T,C,NA,start_lost,"criteria_provided,_multiple_submitters,_no_conflicts",Pathogenic
3,2,c.1A>G,c.1A>G (p.Met1Val),p.Met1Val,p.Met1?,ENSP00000350283.3:p.Met1?,gnomAD and ClinVar,exomes,2,NA,⋯,No evidence*,,17,43124096,T,C,NA,start_lost,"criteria_provided,_multiple_submitters,_no_conflicts",Pathogenic
4,3,c.1A>G,c.1A>G (p.Met1Val),p.Met1Val,p.Met1?,ENSP00000350283.3:p.Met1?,gnomAD and ClinVar,exomes,2,NA,⋯,No evidence*,,17,43124096,T,C,NA,start_lost,"criteria_provided,_multiple_submitters,_no_conflicts",Pathogenic
5,4,c.1A>G,c.1A>G (p.Met1Val),p.Met1Val,p.Met1?,ENSP00000350283.3:p.Met1?,gnomAD and ClinVar,exomes,2,NA,⋯,No evidence*,,17,43124096,T,C,NA,start_lost,"criteria_provided,_multiple_submitters,_no_conflicts",Pathogenic
6,5,c.1A>G,c.1A>G (p.Met1Val),p.Met1Val,p.Met1?,ENSP00000350283.3:p.Met1?,gnomAD and ClinVar,exomes,2,NA,⋯,No evidence*,,17,43124096,T,C,NA,start_lost,"criteria_provided,_multiple_submitters,_no_conflicts",Pathogenic


## Check functional assay classification column

Here we  create a column, called func_class, that specifies the functional assay classification of a variant. It must be in *number* format, as follows:

0 = non-carrier

1 = functional

2 = indeterminate

3 = loss of function

We use the "loss_of_function_threshold" and "functional_threshold" variables we specified earlier in the notebook;

In [12]:
#original code
#input_data$func_class = ifelse(input_data$score >= loss_of_function_threshold, 3,
#                              ifelse(input_data$score <= functional_threshold, 1,
#                                     2))

Here are example function score ranges for classification. These come from the SGE functional assay for BRCA variants, but yours are likely different. 

'functional', score > -0.748; 

'intermediate', -0.748 > score > -1.328; 

'non-functional', score < -1.328. 


You may need to reverse the arrows in the comparison test in the following code if your signs or thresholds are different. 

In [13]:
#adapted and reversed the arrows

#loss_of_function_threshold = -1.328
#functional_threshold = -0.748
input_data$func_class = ifelse(input_data$score <= loss_of_function_threshold, 3,
                              ifelse(input_data$score >= functional_threshold, 1,
                                     2))

Rename your control and case observations columns into "case_obs" and "control_obs"

In [14]:
#reformatting the case control data into simple numbers
input_data <- input_data %>%
    separate(BRIDGES_Case_Carriers_N__pct, into=c("case_fraction", "case_percent"), sep=" ", convert=TRUE) %>%
    separate(case_fraction, into=c("case_obs", "BRIDGES_total_case"), sep="/", convert=TRUE) %>%
    separate(BRIDGES_Control_Carriers_N__pct, into=c("control_fraction", "control_percent"), sep=" ", convert=TRUE) %>%
    separate(control_fraction, into=c("control_obs", "BRIDGES_total_control"), sep="/", convert=TRUE)



Warning message:
“Expected 2 pieces. Missing pieces filled with `NA` in 34266 rows [1, 24, 25,
26, 27, 28, 29, 30, 31, 32, 38, 39, 40, 41, 42, 43, 44, 45, 46, 56, ...].”
Warning message:
“Expected 2 pieces. Missing pieces filled with `NA` in 34266 rows [1, 24, 25,
26, 27, 28, 29, 30, 31, 32, 38, 39, 40, 41, 42, 43, 44, 45, 46, 56, ...].”
Warning message:
“Expected 2 pieces. Missing pieces filled with `NA` in 34266 rows [1, 24, 25,
26, 27, 28, 29, 30, 31, 32, 38, 39, 40, 41, 42, 43, 44, 45, 46, 56, ...].”
Warning message:
“Expected 2 pieces. Missing pieces filled with `NA` in 34266 rows [1, 24, 25,
26, 27, 28, 29, 30, 31, 32, 38, 39, 40, 41, 42, 43, 44, 45, 46, 56, ...].”


## do computations

### sum gnomAD observations and optionally add additional clinvar functional data

We may not use the gnomad control data but here we extract the sum.

Gnomad columns with the allele count always end with ".AC_gnomad". 

In [15]:
test_data_AC <- input_data %>% select(ends_with(".AC_gnomad"))

input_data$gnomad_control_obs = rowSums(input_data, na.rm =T)



### keep unique instances of variants

In [16]:
input_data_unique = input_data %>%
  distinct(VRS_ID, .keep_all = T)

In [17]:
test_data_unique

X,hgvs_nt,txpt_hgvsc_from_ID_clinvar,hgvs_pro_clinvar,hgvs_pro_gnomad,txpt_hgvsp_gnomad,variant_source,set_gnomad,n_alt_alleles_exomes_gnomad,n_alt_alleles_genomes_gnomad,⋯,chr_y,pos_y,ref_y,alt_y,DISTANCE,Consequence,Review_Status,Class,func_class,gnomad_control_obs
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>,<dbl>,<dbl>
0,c.1A>C,c.1A>C (p.Met1Leu),p.Met1Leu,,,ClinVar only,,NA,NA,⋯,NA,NA,,,NA,,,,NA,0
1,c.1A>G,c.1A>G (p.Met1Val),p.Met1Val,p.Met1?,ENSP00000350283.3:p.Met1?,gnomAD and ClinVar,exomes,2,NA,⋯,17,43124096,T,C,NA,start_lost,"criteria_provided,_multiple_submitters,_no_conflicts",Pathogenic,3,3
23,c.1A>T,c.1A>T (p.Met1Leu),p.Met1Leu,,,ClinVar only,,NA,NA,⋯,NA,NA,,,NA,,,,3,0
24,c.2T>A,c.2T>A (p.Met1Lys),p.Met1Lys,,,ClinVar only,,NA,NA,⋯,NA,NA,,,NA,,,,3,0
26,c.2T>C,c.2T>C (p.Met1Thr),p.Met1Thr,,,ClinVar only,,NA,NA,⋯,NA,NA,,,NA,,,,3,0
32,c.2T>G,c.2T>G (p.Met1Arg),p.Met1Arg,,,ClinVar only,,NA,NA,⋯,17,43124095,A,C,NA,start_lost,"criteria_provided,_multiple_submitters,_no_conflicts",Pathogenic,3,0
37,c.3G>A,c.3G>A (p.Met1Ile),p.Met1Ile,,,ClinVar only,,NA,NA,⋯,NA,NA,,,NA,,,,NA,0
44,c.3G>C,c.3G>C (p.Met1Ile),p.Met1Ile,,,ClinVar only,,NA,NA,⋯,NA,NA,,,NA,,,,NA,0
46,c.3G>T,c.3G>T (p.Met1Ile),p.Met1Ile,p.Met1?,ENSP00000350283.3:p.Met1?,gnomAD and ClinVar,exomes,2,NA,⋯,17,43124094,C,A,NA,start_lost,reviewed_by_expert_panel,Pathogenic,3,1


### summarize

In [ ]:
#data_summary = input_data_unique %>%
#  filter(!is.na(func_class)) %>%
#  group_by(func_class) %>%
#  summarise(control_obs = sum(control_obs),
#            case_obs = sum(OBS))


In [18]:
data_summary = input_data_unique %>%
  filter(!is.na(func_class)) %>%
  group_by(func_class) %>%
  summarise(control_obs = sum(control_obs, na.rm =T),
            case_obs = sum(case_obs, na.rm =T))



In [19]:
head(data_summary)

#including clinvar scores

func_class,control_obs,case_obs
<dbl>,<int>,<int>
1,106,133
2,0,5
3,5,77


In [20]:
non_carriers = data.frame(
  func_class = 0,
  control_obs = control - sum(data_summary$control_obs),
  case_obs = case - sum(data_summary$case_obs)
)

data_summary_final = rbind(non_carriers, data_summary)

In [21]:
data_summary_final

func_class,control_obs,case_obs
<dbl>,<dbl>,<dbl>
0,43370,46091
1,106,133
2,0,5
3,5,77


### expand to long table for logistic regression

In [22]:
long_data = data_summary_final %>%
  gather(key = "Category", value = "Count", -func_class) %>%
  rowwise() %>%
  do(data.frame(status = rep(.$Category, .$Count),
                func_class = rep(.$func_class, .$Count)))

### Convert status to a factor
 0 = controls

 1 = cases

In [23]:

long_data$status = ifelse(long_data$status == "control_obs", 0,
                          1)
long_data$status = factor(long_data$status)
long_data$func_class = factor(long_data$func_class)

### glm model

Here we use the glm function to fit a model.

In [24]:
model = glm(status ~ factor(func_class), data = long_data, family = binomial)
summary(model)


Call:
glm(formula = status ~ factor(func_class), family = binomial, 
    data = long_data)

Coefficients:
                    Estimate Std. Error z value Pr(>|z|)    
(Intercept)          0.06085    0.00669   9.096  < 2e-16 ***
factor(func_class)1  0.16606    0.13037   1.274    0.203    
factor(func_class)2  9.50511   32.40637   0.293    0.769    
factor(func_class)3  2.67352    0.46155   5.792 6.94e-09 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 124382  on 89786  degrees of freedom
Residual deviance: 124302  on 89783  degrees of freedom
AIC: 124310

Number of Fisher Scoring iterations: 8


### extract coefficients and standard errors

In [25]:
coef_estimates = summary(model)$coefficients
log_odds = coef_estimates[, "Estimate"]
se = coef_estimates[, "Std. Error"]

### calculate 95% CI

In [26]:

z_value = qnorm(0.975) #1.96 for 95%CI (finds the critical value in the upper tail (0.05/2 = 0.025. 1-0.25 = 0.975))
ci_log_odds_lower = log_odds - z_value * se
ci_log_odds_upper = log_odds + z_value * se

### convert to odds ratios and corresponding CIs

In [27]:
odds_ratios = exp(log_odds)
ci_odds_ratios_lower = exp(ci_log_odds_lower)
ci_odds_ratios_upper = exp(ci_log_odds_upper)

### make new df

In [31]:
or_results = data.frame(
  OR = odds_ratios,
  CI_lower = ci_odds_ratios_lower,
  CI_upper = ci_odds_ratios_upper
)
or_results = or_results %>%
  mutate(across(where(is.numeric), ~round(., 2)))
or_results

,OR,CI_lower,CI_upper
,<dbl>,<dbl>,<dbl>
(Intercept),1.06,1.05,1.080000e+00
factor(func_class)1,1.18,0.91,1.520000e+00
factor(func_class)2,13428.14,0.00,5.156699e+31
factor(func_class)3,14.49,5.86,3.581000e+01


(something on interpreting prop_path maybe?)

In [30]:
prop_path = cbind(data_summary_final, or_results)
prop_path$controls = control
prop_path$cases = case
prop_path$pop_freq = d0
prop_path$prop_threshold = d1
prop_path$prop_path = apply(prop_path, 1, function(row) propallpath(row[2], row[7], row[3], row[8], row[9], row[10]))
prop_path$prop_path = round(prop_path$prop_path, 2)
print(prop_path)

                    func_class control_obs case_obs       OR CI_lower
(Intercept)                  0       43370    46091     1.06     1.05
factor(func_class)1          1         106      133     1.18     0.91
factor(func_class)2          2           0        5 13428.14     0.00
factor(func_class)3          3           5       77    14.49     5.86
                        CI_upper controls cases pop_freq prop_threshold
(Intercept)         1.080000e+00    43481 46306     0.12      0.4054054
factor(func_class)1 1.520000e+00    43481 46306     0.12      0.4054054
factor(func_class)2 5.156699e+31    43481 46306     0.12      0.4054054
factor(func_class)3 3.581000e+01    43481 46306     0.12      0.4054054
                    prop_path
(Intercept)              0.00
factor(func_class)1      0.06
factor(func_class)2      1.00
factor(func_class)3      1.00
